# An exercise in discretisation and the CFL criterion
*These notebooks have been built from Lorena Barba's Computational Fluid Dynamics module. Here we are going to go from a (simple) equation, to a numerical solution of it. We are then going to look at how changing the resolution impacts the speed and validity of the program.*

*Barba, Lorena A., and Forsyth, Gilbert F. (2018). CFD Python: the 12 steps to Navier-Stokes equations. Journal of Open Source Education, 1(9), 21, https://doi.org/10.21105/jose.00021*


## Step 1: 1-D Linear Convection 

The 1-D Linear Convection equation is the simplest, most basic model that can be used to learn something about CFD. 
Here it is, where $w$ is the vertical veolcity and we're using height, $z$, as the vertical coordinate:

$$\frac{\partial w}{\partial t} + c \frac{\partial w}{\partial z} = 0$$

With given initial conditions (understood as a wave), the equation represents the propagation of that initial wave with speed $c$, without change of shape. Let the initial condition be $w(z,0)=w_0(z)$. Then the exact solution of the equation is $w(z,t)=w_0(z-ct)$.

We discretise this equation in both space and time, using the Forward Difference scheme for the time derivative and the Backward Difference scheme for the space derivative. Consider discretising the spatial coordinate $x$ into points that we index from $i=0$ to $N$, and stepping in discrete time intervals of size $\Delta t$.

From the definition of a derivative (and simply removing the limit), we know that:

$$\frac{\partial w}{\partial z}\approx \frac{w(z+\Delta z)-w(z)}{\Delta z}$$

Our discrete equation, then, is:
$$\frac{w_i^{n+1}-w_i^n}{\Delta t} + c \frac{w_i^n - w_{i-1}^n}{\Delta z} = 0 $$

Where $n$ and $n+1$ are two consecutive steps in time, while $i-1$ and $i$ are two neighboring points of the discretized $z$ coordinate. If there are given initial conditions, then the only unknown in this discretization is $w_i^{n+1}$. 

We can solve for our unknown to get an equation that allows us to advance in time, as follows:

$$w_i^{n+1} = w_i^n - c \frac{\Delta t}{\Delta z}(w_i^n-w_{i-1}^n)$$


Now let's try implementing this in Python.

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt    
%matplotlib inline

First, define a few variables... 
(1) Define an evenly spaced grid of points within a spatial domain that is 2 units of length wide, i.e., $z_i\in(0,2)$. 
    (2) define a variable nz, which will be the number of grid points we want and dz will be the distance between any pair of adjacent grid points.

In [ ]:
total_height = 2.0 # height of the model (in m)
dt = 0.025 # dt is the length of each timestep
nz = 41  # define the number of grid points 
dz = total_height / (nz-1) # define the distance between any pair of adjacent grid points (delta z)
nt = 20    #nt is the number of timesteps we want to calculate
c = 1.      #assume wavespeed of c = 1 m/s

Then we need to set up our initial conditions... 
The initial velocity $w_0$ is given as $w = 2$ in the interval $0.5 \leq z \leq 1$ and $w = 1$ everywhere else in $(0,2)$ (i.e., a hat function).

In [ ]:
w_0 = np.ones(nz)      #numpy function zeros()  
w_0[int(.5 / dz):int(1 / dz + 1)] = 2.  #setting w_0 = 1 if 0.5<=z<=1, setting w=0 elswhere
print(w_0) # it shows us a hat function

In [ ]:
# Let's take a look at those initial conditions 
plt.plot(w_0, np.linspace(0, total_height, nz))

Now it's time to implement the discretisation of the convection equation using a finite-difference scheme.
For every element of our array u, we need to perform the operation $$w_i^{n+1} = w_i^n - c \frac{\Delta t}{\Delta z}(w_i^n-w_{i-1}^n)$$

We'll store the result in a new (temporary) array un, which will be the solution $z$ for the next time-step. We will repeat this operation for as many time-steps as we specify and then we can see how far the wave has convected.

(1) Initialise our placeholder array $wn$ to hold the values we calculate for the $n+1$ timestep.
    (2) We have two iterative operations: one in space and one in time (we'll learn differently later), so we'll start by nesting one loop inside the other. Note: when we write: for i in range(1, nz) we will iterate through the w array, but we'll be skipping the first element (the zero-th element).

In [ ]:
wn = np.ones(nz) #Set the velocity as the initial conditions at the beginning of the run
w = w_0.copy()

# In each timestep(25 timesteps in total), iterate through all the grid points...
#...then repeat the iteration for all the timesteps 

for n in range(0, nt):  #loop for values of n from 0 to nt, so it will run nt times
    wn = w.copy() #copy the existing values of w into wn
    for i in range(1, nz): # if starting from the zero-th element, it will crash, due to the value un[i-1] doesn't exist
        w[i] = wn[i] - c * dt / dz * (wn[i] - wn[i-1])  

In [ ]:
# Now let's try plotting our u array after advancing in time.
plt.plot(w_0,np.linspace(0, total_height, nz),label = "initial conditions")
plt.plot(w,np.linspace(0, total_height, nz),label = "At end of run")
plt.legend() 

# Exploring convergence and the CFL criterion

Above we used a grid with 41 points (nz = 41) and a timestep is 0.025 seconds (dt = 0.025). You can see that the "hat" function has not just been pushed upwards (as the analytical solution of the equation suggests should happen). It has also been smoothed out a bit, because of a process called [https://en.wikipedia.org/wiki/Numerical_diffusion|"numerical diffusion"]. This is where the discretisation we used introduces a spurious spreading out of the single. 

The amount of numerical diffusion will depend on the coarseness of our grid. So now, we'll going to experiment with increasing the size of our grid to get a more accurate solution. 

We can do it by defining a new function, so that we can easily examine what happens as we adjust just one variable: the grid size (nz)

In [ ]:
# define a function called 'linearconv()', it allow us to change the number of grid points in over a 2m layer 

def linearconv(nz):
    dz = 2 / (nz - 1)  #dx is the distance between any pair of adjacent grid points
    nt = 20    #nt is the number of timesteps we want to calculate
    dt = .025  #dt is the amount of time each timestep covers 
    c = 1

    w = np.ones(nz)      #defining a numpy array which is nx elements long with every value equal to 1.
    w[int(.5/dz):int(1 / dz + 1)] = 2  #setting u = 2 if 0.5<=x<=1, setting u=1 if 0<x<0.5 or 1<x<2
    w_0=w.copy()
    
    wn = np.ones(nz) #initializing our placeholder array, un, to hold the values we calculate for the n+1 timestep

    for n in range(0, nt):  #iterate through time
        wn = w.copy()  #copy the existing values of u into un
        for i in range(1, nz):
            w[i] = wn[i] - c * dt / dz * (wn[i] - wn[i-1])   # using 1-D linear convection equation
        
    plt.plot(w_0,np.linspace(0, 2, nz),label = "initial conditions")
    plt.plot(w,np.linspace(0, 2, nz),label = "At end of run")
    plt.legend()


Now let's examine the results of our linear convection problem with an increasingly fine mesh

In [ ]:
# Now reproduce the plot in the step 1 and 2 for reference:

linearconv(41) #convection using 41 grid points

In [ ]:
# Increase the number of grid points
# still numerical diffusion present, but it is less severe (curve less smooth).

linearconv(61)

In [ ]:
# the same pattern is present -- the wave is more square than in the previous runs

linearconv(71)

In [ ]:
#completely changed to square curves

linearconv(81)

In [ ]:
linearconv(85)

# This doesn't look anything like our original hat function.

Why does this happen?

In each iteration of our time loop, we use the existing data about our wave to estimate the speed of the wave in the subsequent time step. Initially, the increase in the number of grid points returned more accurate answers. There was less numerical diffusion and the square wave looked much more like a square wave than it did in our first example.

Each iteration of our time loop covers a time-step of length $\Delta t$, which we have been defining as 0.025.
During this iteration, we evaluate the speed of the wave at each of the $z$ points we've created. In the last plot, something has clearly gone wrong.

What has happened is that over the time period $\Delta t$, the wave is travelling a distance which is greater than dz. 

The length dz of each grid box is related to the number of total points nz, so stability can be enforced if the $\Delta t$ step size is calculated with respect to the size of $dz$.

$$\sigma = \frac{c \Delta t}{\Delta z} \leq \sigma_{\max}$$

where $c$ is the speed of the wave; $\sigma$ is called the Courant number and the value of $\sigma_{\max}$ that will ensure stability depends on the kind of discretisation used. Overall this equation is called the CFL criterion. We will use to calculate the appropriate time-step $dt$ depending on the vertical resolution.


In [ ]:
# Re-define the function 'linearconv()' as 'linearconv_CFL(nz)' but make the timestep change dynamically with the grid resolution

def linearconv_CFL(nz):

    dz = 2 / (nz - 1)  #dz is the distance between two adjacent grid points
    run_length = 0.5 # which is the same as before - i.e. 20*0.025
    c = 1
    sigma = .5  # sigma is a Courant number 
    
    dt = sigma * dz  # now, the amount of time that each timestep covers, is calculated with respect to the size of dz...
                            # ...so, stability is enforced (the value of dt now depends on dz)
    nt = int(1 + run_length / dt)
    
    w = np.ones(nz)      #defining a numpy array which is nx elements long with every value equal to 1.
    w[int(.5/dz):int(1 / dz + 1)] = 2  #setting u = 2 if 0.5<=x<=1, setting u=1 if 0<x<0.5 or 1<x<2
    w_0=w.copy()

    wn = np.ones(nz)

    tic = time.perf_counter() #Also add in a
    for n in range(nt):  #iterate through timestep
        wn = w.copy() 
        for i in range(1, nz):
            w[i] = wn[i] - c * dt / dz * (wn[i] - wn[i-1]) 
        
    toc = time.perf_counter()
    time_taken_millisec=(toc-tic)*10e6
    print(f"The model took {time_taken_millisec:0.4f} milliseconds to run")

    plt.plot(w_0,np.linspace(0, 2, nz),label = "initial conditions")
    plt.plot(w,np.linspace(0, 2, nz),label = "At end of run")
    plt.legend()

    return(time_taken_millisec) # return the wallclock time for the model to complete
    

In [ ]:
runtime_nz41=linearconv_CFL(41)

In [ ]:
runtime_nz61=linearconv_CFL(61)

In [ ]:
runtime_nz81=linearconv_CFL(81) 

# Compare to linearconv (41), the number of grid points (nx) doubled (from 41 to 81)... 
# ...which means you have changed to a higher resolution

# The distance between any pair of adjacent grid points (dx) has decreased 1/2 (from 0.05 to 0.025)

# Then, the amount of time each timestep covers (dt) will be changed as well...
# ...it depends on dx and also controlled by the value of sigma (in order to enfore stability)...
# ...so, in this example, dt has decresed 1/2 (from 0.025sec to 0.0125 sec)


# After changing all the variables (nx,dx,dt), iterate through all the grid points in the first timestep...
# ...then do the same iteration for the second timestep....
# ...until complete all the timesteps

In [ ]:
runtime_nz101=linearconv_CFL(101)

In [ ]:
runtime_nz121=linearconv_CFL(121)

### Summary 

Looking all the plots above, you can see that as the number of grid points ($nz$) increases, the convected wave is resolved more accurately (i.e. it becomes more square).

However there is a serious downside to the increasing the resolution - it takes much longer to compute. As the numbers of vertical grid points is increased, it intuitively makes sense that the model will need more computations. For example, as $nz$ increases from 41 to 121 we have tripled the number of gridpoints. So does the time taken for the computation increase by a factor of 3 as well? Let's find out...

In [ ]:
factor=runtime_nz121/runtime_nz41
print(factor)

No, it isn't just a tripling. It is much more than that - closer to a factor of 10.  

The actual you find will depend on what machine you're using to run this notebook and what else is happening on the machine at the time. The reason for this is again down the CFL criterion. 

As we reduced distance between grid points by 1/3, we also needed to reduce the timestep by a similar factor. Therefore the amount of computations has gone up by $3^2$. And that's a theoretical baseline, more computations can run into more inefficiencies and make the run length even longer.

Finally, remember that this is a simple example in 1D. A real climate model is 3D meaning that if you increase the grid resolution by a factor of 3, you'd your number of computations would go up by $3^4$. So you would expect the run to take 81 times as long!  